# Глава 4 Итераторы и генераторы(Python Дэвид Бизли)

Итерации – одна из сильнейших сторон Python. На высшем уровне абстракции вы можете рассматривать итерации как способ обработки элементов последовательности. Однако их возможности намного шире: они включают создание собственных объектов-итераторов, применение полезных паттернов итераций из модуля itertools, создание функций-генераторов и т. д. Эта глава рассматривает типичные задачи, связанные с итерациями. 

#  Ручное прохождение по итератору 

Вам нужно обработать элементы итерируемого объекта, но по какой-то причине вы не хотите использовать цикл. 

Чтобы вручную пройти по итерируемому объекту, используйте функцию next() и напишите код так, чтобы он ловил исключение StopIteration. Например, в этом случае мы вручную читаем строки из файла: 

In [2]:
with open('passwd.txt') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass

one 12345
two 678910

Обычно StopIteration используется для передачи сигнала о конце итерирования. Однако если вы используете next() вручную, вы вместо этого можете запрограммировать возвращение конечного значения, такого как None. Например: 

In [6]:
with open('passwd.txt') as f:
    while True:
        line = next(f, None)
        print(line)
        if line is None:
            break
        
        

one 12345

two 678910
None


В большинстве случаев для прохода по итерируемому объекту используется цикл for. Однако задачи иногда требуют более точного контроля лежащего в основе механизма итераций. Также это полезно, для того чтобы разобраться, как он работает. Следующий интерактивный пример иллюстрирует базовые механизмы того, что происходит во время итерирования: 

In [7]:
items = [1, 2, 3]
# Получаем итератор 
# Вызываем items.__iter__() 
it = iter(items)
# Запускаем итератор 
next(it)
# Вызываем it.__next__() 

1

In [8]:
next(it)

2

In [9]:
next(it)

3

In [10]:
next(it)

StopIteration: 

Последующие рецепты в этой главе раскрывают подробности о приемах итерирования, что предполагает знание базового протокола итераторов. Убедитесь, что этот первый рецепт прочно улегся у вас в памяти. 

# Делегирование итерации 

Вы создали нестандартный объект-контейнер, который внутри содержит список, кортеж или какой-то другой итерируемый объект. Вы хотите заставить ваш новый контейнер работать с итерациями. 

В типичном случае вам нужно определить метод __iter__(), который делегирует итерацию внутреннему содержимому контейнера. Например: 

In [1]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    
#Пример
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


В этой программе метод __iter__() просто перенаправляет запрос на итерацию содержащемуся внутри атрибуту *_children*. 

Протокол итераций Python требует, чтобы __iter__() возвращал специальный объект-итератор, в котором реализован метод __next__(), который и выполняет итерацию. Если вы просто итерируете по содержимому другого контейнера, вам не стоит беспокоиться о деталях внутреннего механизма процесса. Вам нужно просто передать запрос на итерацию. Использование функции iter() здесь позволяет «срезать путь» и написать более чистый код. iter(s) просто возвращает внутренний итератор, вызывая s.__iter__(), – примерно так же, как len(s) вызывает s.__len__(). 

# Создание новых итерационных паттернов с помощью генераторов 

Вы хотите реализовать собственный паттерн итераций, который будет отличаться от обычных встроенных функций (таких как range(), reversed() и т. п.). 

Если вы хотите реализовать новый тип итерационного паттерна, определите его с помощью генератора. Вот, например, генератор, который создает диапазон чисел с плавающей точкой: 

In [1]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

Чтобы использовать такую функцию, вы должны проитерировать по ней в цикле или применить ее с какой-то другой функцией, которая потребляет итерируемый объект (например, sum(), list() и т. п.). Например:

In [2]:
for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [3]:
list(frange(0, 1, 0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

In [4]:
sum(frange(0, 1, 0.125))

3.5

Само присутствие инструкции yield в функции превращает ее в генератор. В отличие от обычной функции, генератор запускается только в ответ на итерацию. Вот эксперимент, который вы можете провести, чтобы понять внутренний механизм работы таких функций: 

In [5]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print('Done!')

In [6]:
# Создает генератор – обратите внимание на отсутствие вывода
c = countdown(3)
c

<generator object countdown at 0x000001E7846A6A98>

In [7]:
# Выполняется до первого yield и выдает значение 
next(c)

Starting to count from 3


3

In [8]:
# Выполняется до следующего yield 
next(c)

2

In [9]:
next(c)

1

In [10]:
next(c)

Done!


StopIteration: 

Ключевая особенность функции-генератора состоит в том, что она запускается только в ответ на операции next в ходе итерирования. Когда генератор возвращает значение, итерирование останавливается. Однако цикл for, который обычно используется для выполнения итераций, сам заботится об этих деталях, поэтому в большинстве случаев вам не стоит волноваться о них. 

# Реализация протокола итератора 

Вы создаете собственные объекты, которые хотите сделать итерируемыми, и ищете простой способ реализовать протокол итератора. 

На текущий момент простейший способ реализации итерируемости в объекте – это использование генератора. В рецепте 4.2 был представлен класс Node, представляющий древовидные структуры. Возможно, вы захотите реализовать итератор, который будет обходить узлы поиском в глубину. Вот как можно это сделать: 

In [15]:
class Node:
    '''Итератор, который обходит узлы поиском в глубину'''
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__ (self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

In [16]:
#Пример
#if __name__ = '__main__':

root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)

child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


В этой программе метод depth_first() может просто прочесть и описать. Сначала он выдает себя, а затем итерируется по каждому потомку, выдавая элементы, производимые методом depth_first() потомка (используя yield from). 

Протокол итератора Python требует __iter__(), чтобы вернуть специальный объект итератора, в котором реализована операция __next__(), а исключение StopIteration используется для подачи сигнала о завершении. Cледующая программа демонстрирует альтернативную реализацию метода depth_first(), использующую ассоциированный класс итератора: 

In [17]:
class NodeA:
    '''Aссоциированный класс итератора'''
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, other_node):
        self._children.append(other_node)
        
    def __iter__ (self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)

In [18]:
class DepthFirstIterator():
    '''Проход на первый уровень в глубину'''
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        
        # Возвращает себя, если только что запущен, создает итератор для потомков 
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        
        # Если обрабатывает потомка, возвращает его следующий элемент 
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
            
         # Переходим к следующему потомку и начинаем итерировать по нему 
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
    

In [19]:
root = NodeA(0)
child1 = NodeA(1)
child2 = NodeA(2)
root.add_child(child1)
root.add_child(child2)

child1.add_child(NodeA(3))
child1.add_child(NodeA(4))
child2.add_child(NodeA(5))

for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


Класс DepthFirstIterator работает так же, как и версия на основе генератора, но он беспорядочен и некрасив, поскольку итератор вынужден хранить много сложной информации о состоянии итерационного процесса. Откровенно говоря, никому не нравится писать такой мозговыносящий код. Реализуйте итератор на базе генератора и успокойтесь на этом. 

#  Итерирование в обратном порядке 

Вы хотите проитерировать по последовательности в обратном порядке. 

Используйте встроенную функцию reversed(). Например: 

In [1]:
#возвращает новый список
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

4
3
2
1


In [3]:
#не меняет а
a.reverse()
a

[4, 3, 2, 1]

Обратная итерация сработает только в том случае, если объект имеет определенный размер или если в нем реализован специальный метод __reversed__(). Если ни одно из этих условий не выполнено, вы должны будете сначала конвертировать объект в список. Например: 

In [6]:
# Выводит файл задом наперед 
f = open('somefile.txt')
for line in reversed(list(f)):
    print(line, end='')
f.close()

passwordtext
6789
12345


 конвертирование итерируемого объекта в список может съесть много памяти, если список получится большим

 итерирование в обратном порядке может быть переопределено в собственном классе, если он реализует метод __reversed__(). 

In [7]:
class Countdown:
    def __init__(self, start):
        self.start = start
        
    def __iter__(self):
        #прямой итератор
        n = self.start
        while n > 0:
            yield n 
            n -= 1 
            
    def __reversed__(self):
        #обратный итератор
        n = 1
        while n <= self.start:
            yield n 
            n += 1 

In [8]:
c = Countdown(10)
list(c)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [9]:
c = Countdown(10)
list(reversed(c))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

#  Определение генератора с дополнительным состоянием 

Вы хотите написать генератор, но функция работает с дополнительным состоянием, которое вам хотелось бы каким-то образом показать пользователю. 

Если вам нужен генератор, который показывает пользователю дополнительное состояние, не забудьте, что вы можете легко реализовать его в форме класса, поместив код генератора в метод __iter__(). Например: 

In [1]:
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
            
    def clear(self):
        self.history.clear()
        

Вы можете обращаться с этим классом так же, как с обычным генератором. Однако, поскольку он создает экземпляр, вы можете обращаться к внутренним атрибутам, таким как history или метод clear(). Например:

In [3]:
with open('somefile.txt') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')

1:12345
2:python
2:python
3:6789
4:python
5:text
6:password
7:python

С генераторами легко попасть в ловушку, если пытаться делать все только с помощью функций. В результате может получиться сложный код, если генератору нужно взаимодействовать с другими частями программы некими необычными способами (раскрытие атрибутов, разрешение на управление через вызов методов и т. п.). В этом случае просто используйте определение класса, как показано выше. Определение генератора в методе __iter__() не изменит ничего в том, как вы напишете алгоритм. Но тот факт, что генератор станет частью класса, упростит задачу предоставления пользователям атрибутов и методов для каких-то взаимодействий. Потенциальная хрупкость показанного приема заключается в том, что он может потребовать дополнительного шага: вызова iter(), если вы собираетесь провести итерацию не через цикл for. Например: 

In [4]:
f = open('somefile.txt') 
lines = linehistory(f)
next(lines)

TypeError: 'linehistory' object is not an iterator

In [5]:
#сначала нужно вызвать iter(), затем итерирование
it = iter(lines)
next(it)

'12345\n'

In [6]:
next(it)

'python\n'

In [7]:
next(it)

'6789\n'

# Получение среза итератора 

Вы хотите получить срез данных, производимых итератором, но обычный оператор среза не работает. 

Функция itertools.islice() отлично подходит для получения срезов генераторов и итераторов. Например: 

In [8]:
def count(n):
    while True:
        yield n
        n += 1

In [9]:
c = count(0)

In [10]:
c[10:20]

TypeError: 'generator' object is not subscriptable

In [11]:
# Теперь используем islice()
import itertools
for x in itertools.islice(c, 10, 20):
    print(x)
    

10
11
12
13
14
15
16
17
18
19


Из итераторов и генераторов получить срез напрямую нельзя, потому что отсутствует информация об их длине (и в них не реализовано индексирование). Результат islice() – это итератор, который создает элементы нужного среза, но делает это путем потребления и выбрасывания всех элементов до стартового индекса среза. Следующие элементы затем производятся объектом islice, пока не будет достигнут конечный индекс среза. Важно отметить, что islice() будет потреблять данные, предоставляемые итератором. Это важно, поскольку итераторы не могут быть отмотаны назад. Если вам нужно возвращаться назад, то вам, наверное, лучше сначала конвертировать данные в список. 

# Пропуск первой части итерируемого объекта 

Вы хотите итерировать по элементам в последовательности, но первые несколько элементов вам неинтересны, и вы хотите их опустить. 

В модуле itertools есть несколько функций, которые могут быть использованы для решения этой задачи. Первая – itertools.dropwhile(). Чтобы использовать ее, вы предоставляете функцию и итерируемый объект. Возвращаемый итератор отбрасывает первые элементы в последовательности до тех пор, пока предоставленная функция возвращает True. А затем выдается вся оставшаяся последовательность.

Предположим, что вы читаете файл, который начинается со строчек с комментариями: 

In [1]:
with open('somefile.txt') as f:
    for line in f:
        print(line, end='')

#12345
#
###python
##6789
python
text
password
python

Если вы хотите пропустить все начальные закомментированные строчки, вот как это можно сделать: 

In [2]:
from itertools import dropwhile
with open('somefile.txt') as f:
    for line in dropwhile(lambda line: line.startswith('#'), f):
        print(line, end='')

python
text
password
python

Этот пример показывает, как можно пропустить первые элементы в соответствии с возвращаемым значением проверочной функции. Если так случилось, что вы знаете точное количество элементов, которые хотите пропустить, то вы можете вместо вышеописанного способа использовать itertools.islice(). Например: 

In [3]:
from itertools import islice
items = ['a', 'b', 'c', 1, 4, 10, 15]
for x in islice(items, 3, None):
    print(x)

1
4
10
15


В этом примере последний аргумент islice() None необходим для того, чтобы обозначить, что вам нужно все за пределами первых трех элементов (а не первые три элемента). То есть срез [3:], а не [:3].

Главное преимущество функций dropwhile() и islice() в том, что они позволяют избежать написания грязного кода наподобие вот такого: 

In [ ]:
with open('somefile.txt') as f:     
    # Пропускаем начальные комментарии     
    while True:         
        line = next(f, '')         
        if not line.startswith('#'):             
            break 
 
    # Обрабатываем оставшиеся строки     
    while line:         
        # Можно заменить полезной обработкой            
        print(line, end='')         
        line = next(f, None)

Отбрасывание первой части итерируемого объекта также немного отличается от простого фильтрования. Например, первая часть этого рецепта может быть переписана вот так: 

In [ ]:
with open('somefile.txt') as f:     
    lines = (line for line in f if not line.startswith('#'))     
    for line in lines:         
        print(line, end='') 

Очевидно, что это отбросит все закомментированные строчки в начале файла, но такое решение отбросит и все остальные такие строчки во всем файле. С другой стороны, решение, которое отбрасывает все элементы до тех пор, пока не будет встречен элемент, не соответствующий условиям отбрасывания, подходит под наши требования: все последующие элементы будут возвращены без фильтрования. Стоит отметить, что этот рецепт работает со всеми итерируемыми объектами, включая те, размер которых нельзя оценить предварительно: генераторами, файлами и другими подобными объектами. 

# Итерирование по всем возможным комбинациям и перестановкам 

Вы хотите проитерировать по всем возможным комбинациям и перестановкам коллекции элементов. 

Модуль itertools предоставляет три функции, подходящие для этой задачи. Первая, itertools.permutations(), принимает коллекцию элементов и создает последовательность кортежей со всеми возможными перестановками (то есть она перемешивает их во всех возможных конфигурациях). Например:

In [2]:
items = ['a', 'b', 'c']

In [3]:
from itertools import permutations
for p in permutations(items):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


Если вы хотите получить все возможные перестановки меньшей длины, то можете передать функции необязательный аргумент со значением длины. Например: 

In [4]:
for p in permutations(items, 2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


Используйте itertools.combinations(), чтобы создать последовательность комбинаций элементов входной последовательности. Например: 

In [5]:
#порядок не важен
from itertools import combinations
for p in combinations(items, 3):
    print(p)

('a', 'b', 'c')


In [6]:
for p in combinations(items, 2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'c')


In [7]:
for p in combinations(items, 1):
    print(p)

('a',)
('b',)
('c',)


Для функции combinations() порядок элементов не имеет значения. Комбинацию ('a', 'b') она считает аналогичной ('b', 'a') – поэтому вторая в выводимых результатах отсутствует.

При создании комбинаций выбранные элементы удаляются из коллекции возможных кандидатов (то есть если 'a' уже выбран, он больше не будет рассматриваться). А функция itertools.combinations_with_replacement() выбирает один и тот же элемент более одного раза. Например: 

In [12]:
from itertools import combinations_with_replacement
items = ['a', 'b', 'с']
for c in combinations_with_replacement(items, 3):
    print(c)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'с')
('a', 'b', 'b')
('a', 'b', 'с')
('a', 'с', 'с')
('b', 'b', 'b')
('b', 'b', 'с')
('b', 'с', 'с')
('с', 'с', 'с')


Этот рецепт показывает лишь небольшую часть мощи модуля itertools. Хотя вы могли бы самостоятельно написать код, который выполняет перестановки и комбинации, это, вероятно, отняло бы у вас больше пары секунд времени. Когда вы сталкиваетесь с нетривиальными задачами в сфере итераций, обратитесь к itertools, это всегда окупается. Если задача распространенная, велик шанс того, что вы найдете готовое решение. 

# Итерирование по парам «индекс–значение» последовательности 

Вы хотите проитерировать по последовательности и при этом хранить информацию о том, какой по счету элемент сейчас обрабатывается. 

Встроенная функция enumerate() изящно справляется с этой задачей:

In [13]:
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx, val)

0 a
1 b
2 c


Для печати вывода с привычными номерами строк (то есть с нумерацией, начинающейся с 1, а не с 0) вы можете передать соответствующий аргумент start:

In [14]:
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list, 1):
    print(idx, val)

1 a
2 b
3 c


Этот прием особенно полезен для учета номеров строк в файлах, если нужно будет вывести номер строки в сообщении об ошибке: 

In [ ]:
def parse_data(filename):
    with open(filename, 'rt') as f:
        for lineno, line in enumerate(f, 1):
            fields = line.split()
            try:
                count = int(fields[1])
                #some code
            except ValueError as e:
                print('Line {}: Parse error: {}'.format(lineno, e))

enumerate() удобна, например, для отслеживания смещения (offset) в списке для вхождений определенных значений. Так что если вы хотите отобразить слова в файле к строчкам, в которых они встречаются, это легко сделать с помощью enumerate() – функция отображает каждое слово на смещение строки в файле, где оно найдено: 

In [17]:
from collections import defaultdict
word_summary = defaultdict(list)

with open('somefile.txt', 'r') as f:
    lines = f.readlines()
    
for idx, line in enumerate(lines):
    #создает список слов в текущей строке
    words = [w.strip().lower() for w in line.split()]
    for word in words:
        word_summary[word].append(idx)

In [20]:
for word in word_summary:
    print(word, word_summary[word])

#12345 [0]
# [1]
###python [2]
##6789 [3]
python [4, 7]
text [5]
password [6]


Если вы выведете word_summary после обработки файла, это будет словарь (default dict, если быть точными), и каждое слово будет ключом. Значение каждого ключа – список номеров строк, где встретилось это слово. Если слово встретилось дважды в одной строке, этот номер строки будет записан в список дважды, что делает возможным получение разнообразных простых метрик текста. 

enumerate() – симпатичное решение для ситуаций, где вы могли бы склоняться к использованию собственной переменной-счетчика. Вы могли бы написать такой код: 

In [ ]:
lineno = 1
for line in f:
    lineno += 1

Но часто более элегантным (и менее подверженным ошибкам) способом становится использование enumerate():


In [ ]:
for lineno, line in enumerate(f):

Значение, возвращаемое функцией enumerate(), является объектом enumerate. Это итератор, который последовательно возвращает кортежи, состоящие из счетчика и значения, возвращаемого вызовом функции next() для последовательности, которую вы обходите. Стоит отметить, что иногда можно запутаться при применении enumerate() к последовательности кортежей, которые при этом распаковываются:


In [ ]:
data = [(1, 2), (3, 4), (5, 6), (7, 8)]
for n, (x, y) in enumerate(data):
    #это верно!

In [ ]:
data = [(1, 2), (3, 4), (5, 6), (7, 8)]
for n, x, y in enumerate(data):
    #это не верно!

# Одновременное итерирование по нескольким последовательностям 

Вы хотите за один раз проитерировать по элементам, содержащимся более чем в одной последовательности. 

Чтобы итерировать по более чем одной последовательности за раз, используйте функцию zip(). Например:


In [1]:
xpts = [1, 5, 4, 2, 10, 7]
ypts = [101, 78, 37, 15, 62, 99]
for x, y in zip(xpts, ypts):
    print(x, y)

1 101
5 78
4 37
2 15
10 62
7 99


zip(a, b) работает путем создания итератора, который производит кортежи (x, y), где x берется из a, а y – из b. Итерирование останавливается, когда заканчивается одна из последовательностей. Поэтому результат будет таким же по длине, как и самая короткая из входных последовательностей. Например: 

In [2]:
a = [1, 2, 3]
b = ['w',  'x', 'y', 'z']
for i in zip(a, b):
    print(i)

(1, 'w')
(2, 'x')
(3, 'y')


Если такое поведение нежелательно, используйте функцию itertools.zip_longest(). Например: 

In [3]:
from itertools import zip_longest
for i in zip_longest(a, b):
    print(i)

(1, 'w')
(2, 'x')
(3, 'y')
(None, 'z')


In [4]:
for i in zip_longest(a, b, fillvalue=0):
    print(i)

(1, 'w')
(2, 'x')
(3, 'y')
(0, 'z')


zip() обычно используется тогда, когда вам нужно создать пары из данных. Предположим, что у вас есть список заголовков столбцов и значения столбцов: 

In [5]:
headers = ['name', 'shares', 'price']
values = ['ACME', 100, 490.1]


Используя zip(), вы можете создать пары значений и поместить их в словарь: 

In [6]:
s = dict(zip(headers, values))

Если вы хотите вывести результат

In [7]:
for name, val in zip(headers, values):
    print(name, '=', val)

name = ACME
shares = 100
price = 490.1


Менее распространенное применение zip() заключается в том, что функции может быть передано не две последовательности, а больше. В этом случае кортежи результата будут иметь такое количество элементов, каким было количество последовательностей. Например: 

In [8]:
a = [1, 2, 3]
b = [10, 11, 12]
c = ['x', 'y', 'z']

for i in zip(a, b, c):
    print(i)

(1, 10, 'x')
(2, 11, 'y')
(3, 12, 'z')


И последнее: важно подчеркнуть, что zip() возвращает итератор. Если вам нужны сохраненные в списке спаренные значения, используйте функцию list(). Например: 

In [9]:
zip(a, b)

In [10]:
list(zip(a, b))

[(1, 10), (2, 11), (3, 12)]

#  Итерирование по элементам, находящимся в отдельных контейнерах 

Вам нужно выполнить одинаковую операцию над большим количеством объектов, но объекты находятся в различных контейнерах, а вам хотелось бы избежать написания вложенных циклов, причем без потери читабельности кода. 

Для упрощения этой задачи можно использовать метод itertools.chain(). Он принимает список итерируемых объектов и возвращает итератор, который скрывает тот факт, что вы на самом деле работаете с несколькими контейнерами. Рассмотрим пример: 

In [11]:
from itertools import chain
a = [1, 2, 3, 4]
b = ['x', 'y', 'z']
for x in chain(a, b):
    print(x)

1
2
3
4
x
y
z


Обычно chain() используется, если вы хотите выполнить некоторые операции над всеми элементами за один раз, но элементы разнесены по разным рабочим наборам. Например:

In [ ]:
# Различные наборы элементов 
active_items = set() 
inactive_items = set() 
 
# Итерируем по всем элементам 
for item in chain(active_items, inactive_items):     
    # Обработка элемента

Это решение намного более элегантно, нежели использование двух отдельных циклов, как показано в этом примере: 

In [ ]:
for item in active_items:     
    # Обработка элемента 
    
for item in inactive_items:     
    # Обработка элемента 

itertools.chain() принимает один или более итерируемых объектов в качестве аргументов. Далее она создает итератор, который последовательно потребляет и возвращает элементы, производимые каждым из предоставленных итерируемых объектов. Это тонкое различие, но chain() эффективнее, чем итерирование по предварительно объединенным последовательностям. Например: 

In [ ]:
# Неэффективно     
for x in a + b: 
    
# Уже лучше     
for x in chain(a, b): 

В первом случае операция a + b создает новую последовательность и дополнительно требует, чтобы a и b относились к одному типу. chain() не выполняет такую операцию, намного эффективнее обращается с памятью, если входные последовательности большие, а также легко применяется к итерируемым объектам различных типов. 

# Создание каналов для обработки данных 

Вы хотите обрабатывать данные итеративно, в стиле обрабатывающего данные канала (похожего на канал Unix – он же конвейер). Например, у вас есть огромный объем данных для обработки, который просто не поместится в память целиком. Решение Генераторы хорошо подходят для реализации обрабатывающих каналов. Предположим, что у вас есть огромный каталог с файлами логов, который вы хотите 

Генераторы хорошо подходят для реализации обрабатывающих каналов. Предположим, что у вас есть огромный каталог с файлами логов, который вы хотите обработать:

foo/     
    access-log-012007.gz     
    access-log-022007.gz     
    access-log-032007.gz     
    ...     
    access-log-012008 
    
bar/     
    access-log-092007.bz2     
    ...     
    access-log-022008

Предположим, каждый файл содержит такие строки данных:


124.115.6.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ..." 200 71

210.212.209.67 - - [10/Jul/2012:00:18:51 -0500] "GET /ply/ ..." 200 11875 

210.212.209.67 - - [10/Jul/2012:00:18:51 -0500] "GET /favicon.ico ..." 404 369 

61.135.216.105 - - [10/Jul/2012:00:20:04 -0500] "GET /blog/atom.xml ..." 304 - ... 

Чтобы обработать эти файлы, вы могли бы создать коллекцию небольших генераторов, которые будут выполнять специфические замкнутые в себе задачи:


In [1]:
import os
import fnmatch
import gzip
import bz2
import re

In [2]:
def gen_find(filepat, top):
    '''Находит все имена  файлов в дереве каталогов, которые совпадают с шаблоном маски оболочки '''
    for path, dirlist, filelist in os.walk(top):
        for name in fnmatch.filter(filelist, filepat):
            yield os.path.join(path, name)

In [3]:
def gen_opener(filenames):
    ''' Открывает последовательность имен файлов, которая раз за разом производит файловый объект.     
    Файл закрывается сразу же после перехода к следующему шагу итерации.'''
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
        else:
            f = open(filename, 'rt')
        yield f
        f.close()

In [4]:
def gen_concatenate(iterators):
    '''Объединяет цепочкой последовательность     
    итераторов в одну последовательность.'''
    for it in iterators:
        yield from it

In [5]:
def gen_grep(pattern, lines):
    ''' Ищет шаблон регулярного выражения    
    в последовательности строк'''
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

Теперь вы можете легко совместить эти функции для создания обрабатывающего канала. Например, чтобы найти все файлы логов, которые содержат слово python, вы можете поступить так: 

In [6]:
lognames = gen_find('access-log*', 'www')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep('(?i)python', lines)
for line in pylines:
    print(line)

Если вы хотите еще расширить функциональность канала, то можете скармливать данные выражениям-генераторам. Например, эта версия находит количество переданных байтов и подсчитывает общую сумму: 

In [ ]:
lognames = gen_find('access-log*', 'www') 
files = gen_opener(lognames) 
lines = gen_concatenate(files) 
pylines = gen_grep('(?i)python', lines) 
bytecolumn = (line.rsplit(None,1)[1] for line in pylines) 
bytes = (int(x) for x in bytecolumn if x != '-') 
print('Total', sum(bytes)) 

In [10]:
"hello worlds again".rsplit(None, 1)

['hello worlds', 'again']

Обработка данных в «каналообразной» манере отлично работает для решения широкого спектра задач: парсинга, чтения из источников данных в реальном времени, периодического опрашивания и т. д. Для понимания представленного выше кода главное – уловить, что инструкция yield действует как своего рода производитель данных для цикла for, который действует как потребитель данных. Когда генераторы соединены, каждый yield скармливает один элемент данных следующему этапу канала, который потребляет его, совершая итерацию. В последнем примере функция sum() управляет всей программой, вытягивая один элемент за другим из канала (конвейера) генераторов.

Приятная особенность этого подхода заключается в том, что каждый генератор является маленьким и замкнутым на себе, поэтому их легко писать и поддерживать. Во многих случаях они получаются настолько универсальными, что могут быть переиспользованы в других контекстах. Получающийся код, который «склеи вает» компоненты вместе, тоже обычно читается как простой для понимания рецепт. 

Есть небольшая тонкость с использованием функции gen_concatenate(). Ее назначение – конкатенировать входные последовательности в одну длинную последовательность строк. itertools.chain() выполняет похожую функцию, но требует, чтобы все объединяемые итерируемые объекты были определены в качестве аргументов. В случае этого конкретного рецепта такой подход потребовал бы инструкции типа lines = itertools.chain(*files), которая заставила бы генератор gen_opener() быть полностью потребленным. Поскольку генератор производит последовательность открытых файлов, которые немедленно закрываются на следующем шаге итерации, chain() использовать нельзя. Показанное решение позволяет решить эту проблему. 

Также в функции gen_concatenate() используется yield from для делегирования субгенератору. Объявление yield from it просто заставляет gen_concatenate() выдать все значения, произведенные генератором it. Это описано далее, в рецепте 4.14. 

И последнее: стоит отметить, что «конвейерный» («канальный») подход не сработает для всех на свете задач обработки данных. Иногда вам просто необходимо работать со всеми данными сразу. Однако даже в этом случае использование каналов генераторов может стать путем логического разбиения задачи. Дэвид Бизли подробно написал об этих приемах в обучающей презентации «Трюки с генераторами для системных программистов»1. Если вам нужны дополнительные примеры, обратитесь к ней. 

http://www.dabeaz.com/generators/Generators.pdf

# Превращение вложенной последовательности в плоскую 

У вас есть вложенная последовательность, и вы хотите превратить ее в один плоский список значений. 

Это легко решается с помощью рекурсивного генератора с инструкцией yield from. Например:

In [8]:
from collections.abc import Iterable

In [9]:
def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

In [10]:
items = [1, 2, [3, 4, [5, 6], 7], 8]

In [11]:
# Производит 1 2 3 4 5 6 7 8 
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


В этой программе isinstance(x, Iterable) просто проверяет, является ли элемент итерируемым объектом. Если это так, то yield from используется в качестве некой подпрограммы, чтобы выдать все его значения. Конечный результат – одна последовательность без вложенности.

Дополнительный аргумент ignore_types и проверка not isinstance(x, ignore_types) нужны для предотвращения определения строк и байтов как итерируемых последовательностей,  разбиения их на отдельные символы. Это позволяет вложенным спискам строк работать так, как большинство людей этого и ожидает: 

In [13]:
items = ['Dave', 'Paula', ['Thomas', 'Lewis']]
for x in flatten(items):
    print(x)

Dave
Paula
Thomas
Lewis


Инструкция yield from – отличный способ написания генераторов, которые вызывают другие генераторы в качестве подпроцедур. Без использования этой инструкции вам придется вставить в код дополнительный цикл. Например: 

In [ ]:
def flatten(items, ignore_types=(str, bytes)):     
    for x in items:         
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):             
            for i in flatten(x):                 
                yield i         
        else:             
            yield x 

Хотя это незначительное изменение, инструкция yield from просто приятнее и делает код чище. Как было отмечено, дополнительная проверка на строки и байты нужна для предотвращения их разбивки на отдельные символы. Если есть еще какие-то типы, которые вы не хотите раскрывать, вы просто можете передать другие значения в ignore_types. Стоит отметить, что yield from играет более важную роль в продвинутых программах, использующих корутины (сопрограммы) и основанную на генераторах многопоточность. См. другой пример в рецепте

# Последовательное итерирование по слитым отсортированным итерируемым  объектам  

У вас есть коллекция отсортированных последовательностей, и вы хотите проитерировать по отсортированной последовательности этих последовательностей, слитых воедино. 

Функция heapq.merge() делает именно это: 

In [3]:
import heapq
a = [1, 4, 7, 10]
b = [2, 5, 6, 11]
for c in heapq.merge(a, b):
    print(c)


1
2
4
5
6
7
10
11


Итеративная природа heapq.merge() подразумевает, что она никогда не читает ни одну из переданных ей последовательностей сразу до конца. Это значит, что вы можете использовать ее на длинных последовательностях с очень незначительным перерасходом ресурсов. Вот, например, как вы можете слить воедино два отсортированных файла: 

In [5]:
import heapq
import pandas as pd


In [7]:
with open('sorted_file_1.txt', 'rt') as file1, \
        open('sorted_file_2.txt', 'rt') as file2, \
        open('merged_file_.txt', 'wt') as outf:
    for line in heapq.merge(file1, file2):
        outf.write(line)


Важно отметить, что heapq.merge() требует, чтобы все передаваемые ей последовательности уже были отсортированы. Она не читает предварительно данные в кучу, не выполняет предварительную сортировку. Также она не выполняет никакой валидации входных данных на соответствие требованиям упорядоченности. Она просто проверяет набор элементов из «голов» каждой переданной последовательности и выдает минимальный из найденных. Далее читается новый элемент из выбранной последовательности, и процесс повторяется до тех пор, пока все входные последовательности не будут полностью потреблены.

# Замена бесконечных циклов while итератором 

У вас есть код, который использует цикл while для итеративной обработки данных, потому что в программе присутствует функция или какое-то необычное проверочное условие, которое нельзя вместить в стандартный итерационный паттерн. 

Вполне обычный код для программ, работающих с вводом-выводом: 

In [8]:
CHUNKSIZE = 8192

In [ ]:
def reader(s):
    while True:
        data = s.recv(CHUNKSIZE)
        #сокет s принимает количество байт равное CHUNKSIZE
        if data == b'':
            break
        process_data(data)
        #process_data - функция обработки данных

Такой код часто можно заменить использованием iter(), как показано ниже: 

In [ ]:
def reader(s):
    for chunk in iter(lambda: s.recv(CHUNKSIZE), b''):
        #функция iter будет возвращать строку/байты размера CHUNKSIZE
        #до тех пор, пока она не пустая
        process_data(data)
        #process_data - функция обработки данных
    
        

Если вы сомневаетесь, будет ли это работать, то можете попробовать похожий пример для обработки файлов: 

In [ ]:
import sys
f = open('/etc/passwd')
for chunk in iter(lambda: f.read(10), ''):
    #из файла считывает 10 символов или байт
    #до тех пор, пока не вернется пустая строка
    n = sys.stdout.write(chunk)
    #выводим chunk в стандартный поток вывода 
    #n - количество байт или символов, которое мы вывели

Малоизвестная возможность встроенной функции iter() заключается в том, что она может опционально принимать вызываемый (callable) аргумент и пороговое значение. При таком использовании функция создает итератор, который снова и снова повторяет вызов предоставленного вызываемого объекта, пока он не вернет значение, равное пороговому значению.

Этот конкретный подход хорошо работает с некоторыми типами многократно вызываемых функций, таких как операции ввода-вывода. Например, если вы хотите читать данные кусочками ("chunk") из файлов или сокетов, то обычно должны многократно вызывать read() или recv() с последующей проверкой достижения конца файла. Представленный выше рецепт просто берет эти две функциональности и совмещает в единственном вызове iter(). Использование lambda в решении необходимо для создания вызываемого объекта, который не принимает аргументов, но при этом поставляет аргумент нужного размера в recv() или read().